In [4]:
from langchain_community.document_loaders import JSONLoader

In [5]:
loader = JSONLoader(
    file_path='sample.json',
    text_content=False,
    jq_schema='.faqs[]')

data = loader.load()

list

In [6]:
for i in range(len(data)):
    print(len(data[i].page_content))

188
194
286
241
272


In [9]:
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

# splitter = RecursiveJsonSplitter(max_chunk_size=300)
splitter = CharacterTextSplitter(chunk_overlap=0, chunk_size=300)

In [10]:
data

[Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 1}, page_content='{"question": "What is Python?", "tags": ["programming", "python", "basics"], "answer": "Python is a high-level, interpreted programming language known for its simplicity and versatility."}'),
 Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 2}, page_content='{"question": "What is Pydantic?", "tags": ["python", "validation", "models"], "answer": "Pydantic is a Python library for data validation and settings management using Python type annotations."}'),
 Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 3}, page_content='{"question": "How do I create a virtual environment in Python?", "tags": ["python", "virtualenv", "environment"], "answer": "You can create a virtual environment in Python by running `python -m venv <env_name>` and activating it using the respective command for your o

In [11]:
texts = splitter.split_documents(data)

In [12]:
texts

[Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 1}, page_content='{"question": "What is Python?", "tags": ["programming", "python", "basics"], "answer": "Python is a high-level, interpreted programming language known for its simplicity and versatility."}'),
 Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 2}, page_content='{"question": "What is Pydantic?", "tags": ["python", "validation", "models"], "answer": "Pydantic is a Python library for data validation and settings management using Python type annotations."}'),
 Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 3}, page_content='{"question": "How do I create a virtual environment in Python?", "tags": ["python", "virtualenv", "environment"], "answer": "You can create a virtual environment in Python by running `python -m venv <env_name>` and activating it using the respective command for your o

In [13]:
import os
from dotenv import load_dotenv

load_dotenv()
COHERE_API = os.environ.get("COHERE_API")
SUPABASE_SERVICE_KEY=os.environ.get("SUPABASE_SERVICE_KEY")
SUPABASE_URL="https://uggjxkgfewdwgxbdivox.supabase.co"
GROQ_API_KEY=os.environ.get("GROQ_API_KEY")



In [14]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(cohere_api_key=COHERE_API,model="embed-english-v3.0")


In [15]:
from supabase.client import Client, create_client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_SERVICE_KEY)

In [28]:
from langchain_community.vectorstores import SupabaseVectorStore
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(
    texts
)
bm25_retriever.k = 2
vector_store = SupabaseVectorStore.from_documents(
    texts,
    embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
)
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5]
)

In [17]:
query = "What does pydantic do?"
matched_docs = vector_store.similarity_search_with_relevance_scores(query)

print(matched_docs[0])



(Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 2}, page_content='{"question": "What is Pydantic?", "tags": ["python", "validation", "models"], "answer": "Pydantic is a Python library for data validation and settings management using Python type annotations."}'), 0.654289083369411)


In [18]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0.0,
    max_retries=2,
)

In [29]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",retriever=ensemble_retriever)

In [30]:
x = qa.run("Tell me about pydantic")

In [31]:
x

'Pydantic is a Python library for data validation and settings management. It uses Python type annotations to ensure that the data being provided meets certain criteria, which can help to catch errors and ensure data consistency. Pydantic models can be used to define the structure and validation rules for incoming data, such as data received from an API or a user input form. This can help to make it easier to work with data in a Python application, as the data will be in a known, validated state.'